In [1]:
            import numpy as np
            import matplotlib.pyplot as plt
            from database_methods import Table, to_show, head
            import sqlite3
            con = sqlite3.connect('database_results.db')
            cursor = con.cursor()

In [29]:
mas = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [166]:
print(mas.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
mas.show_all()
exp.show_all()

CATEGORY, PERIOD, AMPLITUDE, MASTER
DATE, MASTER_ID, DIP, FOLDER
EXPERIMENT_ID, NP, WAVELENGTH, FILE, AR_TH, AR_DIP, AR_NOISE, EXC_THS, RATE, INFO, COUNT, X, Y, CON, INP, INPPX, IBGPX, STD, stdX, stdY, stdCON, stdINP, stdINPPX, stdIBGPX, stdSTD, NOTES, AR_MIN, AR_COND, EXP_TYPE
--------------------------------------------------------------------------------
ID	CATEGORY	PERIOD	AMPLITUDE	MASTER
1	600	405	45	M5
2	600	405	53	P3
3	650	452	32	K4
4	650	452	40	K5
5	750	528	24	L3
6	750	526	30	Q4
ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
4	20-02-26	5	741	20_02_26_L3/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/


In [147]:
# 
columns = 'MASTER,  AVG(DIP), AVG(INP), AVG(INP/IBGPX), AVG(CON), AVG(INPPX), AVG(IBGPX)'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER !='K4'
GROUP BY MASTER
ORDER BY DIP DESC
;
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])


MASTER	 AVG(DI	AVG(INP	AVG(INP	AVG(CON	AVG(INP	AVG(IBG
Q4	727.00	27.87	62.55	3.87	1.77	0.4825
K5	658.00	14.79	48.99	5.44	1.64	0.3078
P3	615.00	5.25	20.50	7.27	1.81	0.2621


In [224]:
# 
columns = ' MASTER, FILE, WAVELENGTH, NP, DIP, COUNT, RATE, CON, COUNT, INP, IBGPX, STD, EXP_TYPE,  NOTES'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE CATEGORY = 750
ORDER BY CON DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

 MASTER	FILE	WAVELEN	NP	DIP	COUNT	RATE	CON	COUNT	INP	IBGPX	STD	EXP_TYP	 NOTES
Q4	raw_05_	734	80	727	169	70.29	5.38	169	36.50	0.5129	0.6644	-	-
Q4	raw_04_	732	80	727	206	85.33	4.42	206	46.00	0.5202	0.6898	-	-
L3	raw_31_	743	100	740	54	14.52	4.37	54	13.20	0.3453	0.4741	-	-
Q4	raw_18_	732	60	727	166	97.04	3.97	166	31.03	0.5038	0.6527	-	-
L3	raw_11_	741	80	740	80	31.72	3.92	80	7.49	0.2188	0.2834	-	FPs
L3	raw_09_	748	80	740	67	24.06	3.91	67	22.18	0.4542	0.5750	-	-
Q4	raw_21_	732	60	727	234	96.97	3.82	234	30.71	0.4713	0.6189	sh	-
Q4	raw_07_	740	80	727	128	59.20	3.75	128	15.91	0.5354	0.6807	-	-
L3	raw_30_	745	100	740	74	23.77	3.75	74	14.86	0.4289	0.5891	-	FPs	be
L3	raw_12_	740	80	740	83	36.96	3.70	83	19.31	0.1997	0.2547	-	nice
L3	raw_07_	745	80	740	94	29.50	3.66	94	33.34	0.4178	0.5305	-	nice
Q4	raw_08_	726	80	727	113	70.91	3.64	113	20.36	0.2148	0.2813	-	-
L3	raw_15_	737	80	740	67	24.29	3.43	67	12.51	0.3739	0.4733	-	bg nois
L3	raw_08_	746	80	740	86	24.14	3.36	86	31.20	0.4340	0.5498	-	nice
L3	r

In [220]:
columns = 'NP, WAVELENGTH, AR_TH, AR_NOISE, EXC_THS'
data = con.execute("""
SELECT {} FROM 'measurements'
WHERE WAVELENGTH=622
ORDER BY WAVELENGTH ASC;
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	AR_TH	AR_NOISE	EXC_THS
80	622	3.0	0.0014	[4	0	1.1]
80	622	2.5	0.001	[2]


In [213]:
con.execute("""
UPDATE 'measurements' SET NOTES = 'weak' WHERE COUNT = 165
""")

In [215]:
meas.commit()

In [9]:
con.close()